### Create Patent Random Sample
* once I have pre processed all the patents in the sample, create an index file listing those with missing info
* this is what the random sample will come from
* then, I need to upload the claims code to engaging so it can be easily run for all patents in the random sample
* Once that is completed, I will generate a random sample of those patents 
* I need to update the description and references code to add an append option - so I don't duplicate work
* then, I can do a basic classifier

* On the sloan partition, I also need to test the threading and multiprocessing

In [31]:
import concurrent
import pickle
import json
import funcy
import csv
import os
import gzip
import pandas as pd
import numpy as np
import imblearn

import sklearn
# from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.pipeline import make_pipeline


from imblearn.under_sampling import RandomUnderSampler
# #from imblearn.pipeline import make_pipeline as make_pipeline_imb
# from imblearn.metrics import classification_report_imbalanced



In [3]:
# Split description directory
index_file = '/pool001/lraymond/patent_data/index_files/pat_nums_index.csv'
PATENTS_PICKLE_DIR = '/nobackup1/lraymond/patent_data/numerical_patents_datasets/dataframes/stored_pickles'


In [ ]:
def is_valid_patent_number(patent_num):
    '''try to convert patent number to int, flag errors'''
    try:
        t = int(patent_num)
        return True
    except (TypeError, ValueError):
        return False

In [ ]:
def subset_patents(filename, patents_pickle_dir=PATENTS_PICKLE_DIR):
    df = pd.read_pickle(os.path.join(patents_pickle_dir, filename))
    df2 = df[['10_year_cites', '15_year_cites', '20_year_cites', '30_year_cites',
       '5_year_cites', 'assignee_is_company', 'assignee_is_gov',
       'assignee_is_ind', 'forprior_country',
       'forprior_date', 
       'inventor_total_num_patents', 'lawyer_organization',
       'lawyer_total_num_assignees', 'lawyer_total_num_inventors',
       'lawyer_total_num_patents', 
       'nber_category_id',  'nber_subcategory_id',
        'number_apps', 'number_assignees',
        'number_assistant_examiners',
       'number_examiners', 'number_forprior', 'number_govint',
       'number_inventors',  'number_lawyers',
       'number_primary_examiners', 'patent_abstract', 'patent_date',
      'patent_firstnamed_assignee_country',
        'patent_firstnamed_assignee_state',
        'patent_firstnamed_inventor_country',
        'patent_firstnamed_inventor_state',
        'patent_num_cited_by_us_patents',
       'patent_num_combined_citations', 'patent_num_foreign_citations',
       'patent_num_us_application_citations', 'patent_num_us_patent_citations',
       'patent_number', 'patent_title',  'patent_year',
       '10_year_cites_rank', '10_year_cites_top1', '15_year_cites_rank',
       '15_year_cites_top1', '20_year_cites_rank', '20_year_cites_top1',
       '30_year_cites_rank', '30_year_cites_top1', '5_year_cites_rank',
       '5_year_cites_top1', 'flag_has_forprior',
       'patent_firstnamed_assignee_latitude',
       'patent_firstnamed_assignee_longitude',
       'patent_firstnamed_inventor_latitude',
       'patent_firstnamed_inventor_longitude', 
        'missing_citations_made', 'missing_patent_abstract',
       'missing_patent_title']].sort_values('patent_number').drop_duplicates('patent_number')
    df2['is_valid_patent_number'] = df2.patent_number.apply(is_valid_patent_number)
    # currently using only patents with abstract, title and citations
    mask = (
        (df2.missing_citations_made==0) &(
                df2.missing_patent_abstract==0) & (
                    df2.missing_patent_abstract==0) & (df2.is_valid_patent_number==1))
    return df2[mask]
    

In [ ]:
# Open up the patent dataframes from each year, create an index file with those with citations made, 
txt_files = sorted([f for f in os.listdir(
    patents_pickle_dir) if os.path.isfile(os.path.join(patents_pickle_dir, f))])

In [ ]:
valid_patents = pd.concat(list(map(subset_patents, txt_files)), axis=0)

In [ ]:
valid_patents2 = valid_patents.sort_values(['patent_number', 'patent_year']).drop_duplicates()

In [48]:
valid_patents2.shape

(1994958, 59)

In [49]:
valid_patents2[['assignee_is_company', 'assignee_is_gov',
       'assignee_is_ind',]] = valid_patents2[['assignee_is_company', 'assignee_is_gov',
       'assignee_is_ind',]].fillna(0).astype(int, errors='ignore')

In [ ]:
def convert_assignee_is_ind(val):
    if pd.isnull(val):
        return np.nan
    if isinstance(val, str):
        return int(val=='True')
    try:
        return int(val)
    except ValueError:
        print(val)
        return 0
        

valid_patents2['assignee_is_ind'] = valid_patents2['assignee_is_ind'].apply(convert_assignee_is_ind)

valid_patents2.to_csv(index_file, index=False)

In [50]:
valid_patents2 = pd.read_csv(index_file)

/home/software/sloan/local/lib/py36/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
valid_patents2.head()